# Analogous stats

In [ ]:
using Pkg
Pkg.activate("../../env/.")

In [1]:
include("../../src/analogous_bars.jl")
include("../../src/Eirene_var.jl")
include("../../src/eirene_helper.jl")
using .analogous_bars
using .Eirene_var
using .eirene_helper
using DelimitedFiles
using HDF5
using Plots
using FileIO
using Random


WebIO._IJuliaInit()

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /opt/julia/packages/ORCA/U5XaN/src/ORCA.jl:8


In [3]:
using Distributed
#addprocs(5)
#print(nprocs());

@everywhere include("../../src/analogous_stats.jl")
@everywhere using .analogous_stats

## 1. Encoding: stimulus & V1

Load dissimilarity matrices

In [5]:
directory = "data"

# load distance matrices
D_stimulus = readdlm(directory * "/stimulus_perturbed_distance.csv")
D_V1 = h5read(directory * "/V1_simulation_distance.h5","distance")
D_V1 = vector_to_symmetric_matrix(D_V1, 800);

D_V1_stimulus = Array(transpose(h5read(directory * "/V1_stimulus_distance.h5", "distance")));
D_stimulus_V1 = Array(transpose(D_V1_stimulus));

Compute VR persistence

In [6]:
dim = 1

# VR persistence
VR_stimulus = eirene(D_stimulus, record = "all", maxdim = dim)
VR_V1 = eirene(D_V1, record = "all", maxdim = dim);

Load the VR, W persistence diagrams and cutoffs

In [ ]:

#save(directory * "/stimulus_V1_baseline_similarity_analogous.jld2", 
#    "barcodes", barcodes,
#    "persistence_cutoff", persistence_cutoff,
#    "analogous_pair", analogous_pair)

In [7]:
outputs = load("data/stimulus_V1_baseline_similarity_analogous.jld2")

# load barcodes
barcode_V1 = outputs["barcodes"]["VR_V1"]
barcode_stimulus = outputs["barcodes"]["VR_stimulus"]
barcode_W_V1_stimulus = outputs["barcodes"]["W_V1_stimulus"]

# load cutoffs
V1_cutoff = outputs["persistence_cutoff"]["V1_cutoff"]
stimulus_cutoff = outputs["persistence_cutoff"]["stimulus_cutoff"]
W_cutoff = outputs["persistence_cutoff"]["W_V1_stimulus_cutoff"];

In [8]:
selected_V1 = findall(x -> x > V1_cutoff, barcode_V1[:,2] - barcode_V1[:,1])
selected_stimulus = findall(x -> x > stimulus_cutoff, barcode_stimulus[:,2] - barcode_stimulus[:,1])

3-element Array{Int64,1}:
 275
 276
 277

In [ ]:
# script to compute from 50 to 100
start_idx = collect(151:5:200)
for idx in start_idx
    shuffles = collect(idx:idx+4) 
    t = @distributed for k in shuffles
        output_filename = "null_model_stats/stimulus_V1/" * string(k)
        _, _, _, _, _ = analogous_stats.compute_analogous_bars_null_model(;
                                                                        VR_P = VR_stimulus,
                                                                        D_P = D_stimulus,
                                                                        VR_Q = VR_V1,
                                                                        D_Q = D_V1,
                                                                        P_null_dim = 3,
                                                                        Q_null_dim = 3,
                                                                        output_filename = output_filename
                                                                        )

    end
    fetch(t)
end

      From worker 2:	400
      From worker 2:	800
      From worker 3:	400
      From worker 3:	800
      From worker 4:	400
      From worker 4:	800
      From worker 5:	400
      From worker 5:	800
      From worker 6:	400
      From worker 6:	800


In [ ]:
fetch(t)

In [18]:
n_shuffles = 200
output_dir = "null_model_stats/stimulus_V1/"
stats_matrix, stats_total, selected_bar_count, no_Ftau_P_or_Q, valid_files = parse_null_model_stats(output_dir, 
                                                                         barcode_stimulus, 
                                                                         barcode_V1,
                                                                         selected_stimulus,
                                                                         selected_V1,
                                                                         n_shuffles = n_shuffles)

prob_matrix = stats_matrix ./ n_shuffles

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [21]:
# count the total number of bars from which we ran similarity_centric_analogous_bars
output_dir = "null_model_stats/grid_conj/"
for i = 1:10


    # open julia files
    analogous_P = load(output_dir * string(i) * ".jld2")["analogous_P"]
    analogous_Q = load(output_dir * string(i) * ".jld2")["analogous_Q"]
    selected = load(output_dir * string(i) * ".jld2")["W_selected"]

    println(selected)
    println(analogous_P)
    println(analogous_Q)
end

[48, 50]
Dict{Any,Any}(50 => [1],48 => [8, 1])
Dict{Any,Any}(50 => nothing,48 => nothing)
[44, 45]
Dict{Any,Any}(44 => nothing,45 => nothing)
Dict{Any,Any}(44 => nothing,45 => nothing)
[55, 56]
Dict{Any,Any}(55 => nothing,56 => nothing)
Dict{Any,Any}(55 => nothing,56 => nothing)
[55, 56]
Dict{Any,Any}(55 => nothing,56 => nothing)
Dict{Any,Any}(55 => nothing,56 => nothing)
[44, 46]
Dict{Any,Any}(44 => nothing,46 => nothing)
Dict{Any,Any}(44 => nothing,46 => nothing)
[54, 55]
Dict{Any,Any}(54 => nothing,55 => [3])
Dict{Any,Any}(54 => nothing,55 => nothing)
[49, 51]
Dict{Any,Any}(49 => [8],51 => nothing)
Dict{Any,Any}(49 => nothing,51 => nothing)
[42, 43]
Dict{Any,Any}(43 => nothing,42 => nothing)
Dict{Any,Any}(43 => nothing,42 => nothing)
[44, 47]
Dict{Any,Any}(47 => [2],44 => nothing)
Dict{Any,Any}(47 => nothing,44 => nothing)
[46, 47]
Dict{Any,Any}(47 => nothing,46 => nothing)
Dict{Any,Any}(47 => nothing,46 => nothing)


# 2. V1 & orientation

Load dissimilarity matrices

In [4]:
# load distances
directory = "data"
V1_distance = h5read(directory * "/V1_simulation_distance.h5", "distance")
O_distance = h5read(directory * "/orientation_distance.h5", "distance")
D_ori_V1 = h5read(directory * "/V1_orientation_distance.h5", "distance") # this is probably orientation_V1 

# number of neurons
n_V1 = 800
n_O = 64


# convert vector of distances to distance matrix 
D_V1 = vector_to_symmetric_matrix(V1_distance, n_V1)
D_O = vector_to_symmetric_matrix(O_distance, n_O)
D_V1_ori = Array(transpose(D_ori_V1));

Compute VR persistence

In [5]:
VR_O = eirene(D_O, record = "all", maxdim = 1)
VR_V1 = eirene(D_V1, record = "all", maxdim = 1);

Load the VR, W persistence diagrams and cutoffs

In [6]:
outputs = load("data/V1_orientation_baseline_similarity_analogous.jld2")

# load barcodes
barcode_O = outputs["barcodes"]["VR_orientation"]
barcode_V1 = outputs["barcodes"]["VR_V1"]
barcode_W = outputs["barcodes"]["W_orientation_V1"]

# load cutoffs
O_cutoff = outputs["persistence_cutoff"]["ori_cutoff"]
V1_cutoff = outputs["persistence_cutoff"]["V1_cutoff"]
W_cutoff = outputs["persistence_cutoff"]["W_ori_V1_cutoff"]

# selected bars
selected_O = outputs["persistence_cutoff"]["ori_selected"]
selected_V1 = outputs["persistence_cutoff"]["V1_selected"]
selected_W = outputs["persistence_cutoff"]["W_ori_V1_selected"];

In [ ]:
# script to compute from 50 to 100
start_idx = collect(146:5:201)
for idx in start_idx
    shuffles = collect(idx:idx+4) 
    t = @distributed for k in shuffles
        output_filename = "null_model_stats/V1_ori/" * string(k)
        _, _, _, _, _ = analogous_stats.compute_analogous_bars_null_model(;
                                                                        VR_P = VR_V1,
                                                                        D_P = D_V1,
                                                                        VR_Q = VR_O,
                                                                        D_Q = D_O,
                                                                        P_null_dim = 3,
                                                                        Q_null_dim = 1,
                                                                        output_filename = output_filename
                                                                        )

    end
    fetch(t)
end

In [7]:
n_shuffles = 200
output_dir = "null_model_stats/V1_ori/"

stats_matrix, stats_total, selected_bar_count, no_Ftau_P_or_Q, valid_files = parse_null_model_stats(output_dir, 
                                                                         barcode_V1, 
                                                                         barcode_O,
                                                                         selected_V1,
                                                                         selected_O,
                                                                         n_shuffles = n_shuffles)

prob_matrix = stats_matrix ./ n_shuffles

3×1 Array{Float64,2}:
 0.0
 0.0
 0.0

In [11]:
no_Ftau_P_or_Q

0

# 3. V1 & direction

Load dissimilarity matrices

In [12]:
# load distances
directory = "data"
V1_distance = h5read(directory * "/V1_simulation_distance.h5", "distance")
dir_distance = h5read(directory * "/direction_distance.h5", "distance")
D_dir_V1 = h5read(directory * "/V1_direction_distance.h5", "distance") # this is probably orientation_V1 

# number of neurons
n_V1 = 800
n_D = 64


# convert vector of distances to distance matrix 
D_V1 = vector_to_symmetric_matrix(V1_distance, n_V1)
D_dir = vector_to_symmetric_matrix(dir_distance, n_D)
D_V1_dir = Array(transpose(D_dir_V1));

Compute VR persistence

In [13]:
VR_D = eirene(D_dir, record = "all", maxdim = 1)
VR_V1 = eirene(D_V1, record = "all", maxdim = 1);

Load the VR, W persistence diagrams and cutoffs

In [14]:
outputs = load("data/V1_direction_baseline_similarity_analogous.jld2")

# load barcodes
barcode_dir = outputs["barcodes"]["VR_direction"]
barcode_V1 = outputs["barcodes"]["VR_V1"]
barcode_W = outputs["barcodes"]["W_direction_V1"]

# load cutoffs
dir_cutoff = outputs["persistence_cutoff"]["direction_cutoff"]
V1_cutoff = outputs["persistence_cutoff"]["V1_cutoff"]
W_cutoff = outputs["persistence_cutoff"]["W_direction_V1_cutoff"]

# selected bars
selected_dir = outputs["persistence_cutoff"]["direction_selected"]
selected_V1 = outputs["persistence_cutoff"]["V1_selected"]
selected_W = outputs["persistence_cutoff"]["W_direction_V1_selected"];

In [6]:
nprocs()

6

In [ ]:
# script to compute from 50 to 100
start_idx = collect(151:5:201)
for idx in start_idx
    shuffles = collect(idx:idx+4) 
    t = @distributed for k in shuffles
        output_filename = "null_model_stats/V1_dir/" * string(k)
        _, _, _, _, _ = analogous_stats.compute_analogous_bars_null_model(;
                                                                        VR_P = VR_V1,
                                                                        D_P = D_V1,
                                                                        VR_Q = VR_D,
                                                                        D_Q = D_dir,
                                                                        P_null_dim = 3,
                                                                        Q_null_dim = 1,
                                                                        output_filename = output_filename
                                                                        )

    end
    fetch(t)
end

In [ ]:
interrupt()

In [15]:
n_shuffles = 200
output_dir = "null_model_stats/V1_dir/"

stats_matrix, stats_total, selected_bar_count, no_Ftau_P_or_Q, valid_files = parse_null_model_stats(output_dir, 
                                                                         barcode_V1, 
                                                                         barcode_dir,
                                                                         selected_V1,
                                                                         selected_dir,
                                                                         n_shuffles = n_shuffles)

prob_matrix = stats_matrix ./ n_shuffles

3×1 Array{Float64,2}:
 0.0
 0.0
 0.0

In [37]:
maximum(stats_total)

0.0